# Лабораторная работа 3. Линейные методы.

## Линейная регрессия

В рамках данного задания мы будем работать с датасетом [*bikes_rent.csv*](./bikes_rent.csv), в котором для каждого дня имеется календарная информация и погодные условия, характеризующие автоматизированные пункты проката велосипедов, а также число прокатов велосипедов в этот день в автоматизированных пунктах . Последнее мы будем предсказывать; таким образом, мы будем решать задачу регрессии.

Загрузите данные при помощи функции *pandas.read_csv*. Выведите первые 5 строк, чтобы убедиться в корректном считывании данных.

Для каждого дня проката известны значения следующих признаков:
* _season_: 1 - весна, 2 - лето, 3 - осень, 4 - зима
* _yr_: 0 - 2011, 1 - 2012
* _mnth_: от 1 до 12 (соответственно январь - декабрь)
* _holiday_: 0 - нет праздника, 1 - есть праздник
* _weekday_: от 0 до 6 (соответственно понедельник - воскресенье)
* _workingday_: 0 - нерабочий день, 1 - рабочий день
* _workthersit_: оценка благоприятности погоды от 1 (чистый, ясный день) до 4 (ливень, туман)
* _temp_: температура в Цельсиях
* _atemp_: температура по ощущениям в Цельсиях
* _hum_: влажность
* _windspeed(mph)_: скорость ветра в милях в час
* _windspeed(ms)_: скорость ветра в метрах в секунду
* _cnt_: количество арендованных велосипедов (это целевой признак, его мы будем предсказывать)

Итак, у нас есть вещественные, бинарные и номинальные (порядковые) признаки, и со всеми из них можно работать как с вещественными. 

Сохраните в отдельную переменную матрицу объект-признак и в другую — столбец с целевым признаком.

**(0.5 балла)** В начале попробуем посмотреть на признаки. Постройте матрицу коэффициентов корреляции Пирсона между признаками. Для этого можно воспользоваться методом датафрейма *corr*. Для удобства можно визуализировать ее как *hot-map* (пример, как это можно сделать, есть в семинаре про визуализацию). Можно ли сказать, что среди признаков есть линейно зависимые? Укажите какие?

**(0.5 балла)** Выведите средние значения всех признаков. Можно ли сказать, что признаки имеют разный масштаб? В случае положительного ответа стандартизуйте признаки (из признака вычитается среднее и делится на стандартное отклонение), применив функцию [scale](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.scale.html). 

**(1 балл)** Теперь попробуем обучить модель регрессии. Линейные модели реализованы в модуле *sklearn.linear_model*. Создайте объект [LinearRegression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) с параметрами по умолчанию, обучите его на всех данных X и ответах y, а затем для каждого признака выведите его название (хранятся в после *columns* исходного датафрейма) и вес (веса хранятся в переменной *coef_* класса регрессора).

Обратите внимание, что веса при линейно-зависимых признаках по модулю значительно больше, чем при других признаках.

Чтобы понять, почему так произошло, вспомним аналитическую формулу, по которой вычисляются веса линейной модели в методе наименьших квадратов:

$w = (X^TX)^{-1} X^T y$.

Если в X есть линейно-зависимые столбцы, матрица $X^TX$ становится вырожденной, и формула перестает быть корректной. Чем более зависимы признаки, тем меньше определитель этой матрицы и тем хуже аппроксимация $Xw \approx y$. Такую ситуацию называют _проблемой мультиколлинеарности_.

__Решение__ проблемы мультиколлинеарности состоит в _регуляризации_ линейной модели. К оптимизируемому функционалу $||Xw-y||^2$ прибавляют L1 $\left(\sum_{j=1}^d |w_j| \right)$ или L2 норму весов $\left(\sum_{j=1}^d w_j^2 \right)$, умноженную на коэффициент регуляризации $\alpha$. В первом случае метод называется Lasso, а во втором — Ridge.

**(2 балла)** Обучите регрессоры [Lasso](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) и [Ridge](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html) с параметрами по умолчанию, выведите значения весов аналогично тому как это делалось ранее и убедитесь, что описанная проблема решена. 

В отличие от L2-регуляризации, L1 производит отбор признаков путем обнуления весов при некоторых признаках. Объяснение последнему факту было дано на лекции.

Давайте пронаблюдаем, как меняются веса при увеличении коэффициента регуляризации $\alpha$. 

**(1 балл)** Для каждого значения коэффициента регуляризации из *alphas* создайте регрессор Lasso, указав в качестве параметра соответствующее значение alpha, обучите его на объектах X и ответах y и запишите полученные веса в соответствующую строку матрицы coefs_lasso,  а затем проделайте то же самое для регрессора Ridge и переменной coefs_ridge.

In [ ]:
alphas = np.arange(1, 500, 50)
coefs_lasso = np.zeros((alphas.shape[0], X.shape[1])) # матрица весов размера (число регрессоров) x (число признаков)
coefs_ridge = np.zeros((alphas.shape[0], X.shape[1]))

Визуализируйте динамику весов при увеличении параметра регуляризации, вызвав следующий код:

In [ ]:
plt.figure(figsize=(8, 5))
for coef, feature in zip(coefs_lasso.T, df.columns):
    plt.plot(alphas, coef, label=feature, color=np.random.rand(3))
plt.legend(loc="upper right", bbox_to_anchor=(1.4, 0.95))
plt.xlabel("alpha")
plt.ylabel("feature weight")
plt.title("Lasso")

plt.figure(figsize=(8, 5))
for coef, feature in zip(coefs_ridge.T, df.columns):
    plt.plot(alphas, coef, label=feature, color=np.random.rand(3))
plt.legend(loc="upper right", bbox_to_anchor=(1.4, 0.95))
plt.xlabel("alpha")
plt.ylabel("feature weight")
plt.title("Ridge")

**(2 балла)** Ответьте на вопросы ниже.

1. Какой регрессор (Ridge или Lasso) агрессивнее уменьшает веса?
1. Что произойдет с весами Lasso, если alpha сделать очень большим? Поясните, почему так происходит.
1. Можно ли утверждать, что Lasso исключает один из признаков windspeed при любом значении alpha? А Ridge?
1. Какой из регрессоров подойдет для отбора неинформативных признаков?

## Линейная классификация

В этой части работы мы познакомимся с датасетом MNIST, объектами которого являются рукописные изображени цифр. Каждый объект представляет из себя матрицу размером 8х8 (в данных это выглядит как вектор с 64 признаками). Каждый признак — оттенок серого (в шкале от 0 до 16). Например, при визуализации это будет выглядеть примерно следующим образом (каждый квадратик — отдельный объект):

![](https://www.tensorflow.org/versions/r0.8/images/mnist_digits.png)

Загрузите данные с помощью функции [load_digits](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_digits.html).

У загруженного датасета целевая переменная находится в поле *target*, а сами данные в поле *data*. 

**(0.2 балла)** Посчитайте сколько различных классов цифр есть в данных, а также сколько в каждом классе содержится объектов.

Для оценивания качества алгоритма будем использовать отложенную выборку. Воспользуйтесь функцией [train_test_split](http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.train_test_split.html) чтобы разделить данные на обучающие и тестовые. Важно, чтобы от перезапуска ваше разбиение не менялось, кроме того соотношение классов между собой соранялось. Для этого вам понадобится установить следующие дополнительные значения у данной функции:
 - *test_size* или *train_size* (любой на выбор) — установите параметр таким образом, чтобы в тестовую выборку попала половина всех примеров
 - *random_state* — необходимо задать некоторое число (любое на ваш выбор), чтобы разбиение оставалось фиксированным не зависимо от запуска
 - *stratify* — необходимо задать, чтобы соотношение классов в исходной выборке и новых совпадало; для этого нужно передать значение поля *target*

Для начала будем решать задачу бинарной классификации, а именно попробуем отделить цифру 8 от всех остальных. Создайте новую переменную, которая будет аналогично переменной *target* (своя для обучающей и тестовой выборки), в которой будет стоять 1, если соответствующая метка "8" и -1 в остальных случаях.

**(0.8 балла)** Сколько теперь получилось объектов положительного класса, а сколько — отрицательного? Какая метрика качества из изученных ранее подойдет для данной задачи и почему? Чем плохо использовать, например, *Accuracy*?

Рассмотрим линейный классификатор: $$ a(x) = sign \langle w, x\rangle $$

Обычно вводят обозначение $M_i = y_i\langle w, x_i \rangle$ — отступ на объекте $x_i$

 - $M_i < 0$ — классификатор ошибся на объекте $x_i$
 - $M_i > 0$ — классификатор не ошибся на объекте $x_i$
 
Целью классификации является минимизиция количества ошибок классификатора, то есть

$Q(w) = \sum_{i=1}^l[M_i(w) < 0] \to \min_{w}$

Это выражение достаточно сложно оптимизировать, поэтому вводять *аппроксимацию*:

$Q(w) = \sum_{i=1}^l[M_i(w) < 0] \le \tilde{Q}(w)$

В качестве $\tilde{Q}$ можно рассмотреть следующие функции:
![](http://i.imgur.com/z12WF2C.png)
 - *Zero-one loss* — исходная функция 
 - *Log loss* — экспоненциальная (используется в логистической регрессии)

Логистическая регрессия реализована в sklearn в классе [LogisticRegression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html). Одним из ее гиперпараметров является коэффициент регуляризации *C*. Как вы помните, гиперпараметры подбираются с помощью кросс-валидации. 

Для начала зафиксируем схему кросс-валидации: будем использовать кросс-валидацию по 5 блокам. Для этого создайте объект [KFold](http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.KFold.html), которому необходимо задать размер выборки (обучающей), а также задать следующие параметры:
 - *n_folds* — число фолдов (5 в данном случае)
 - *random_state* — аналогично как было выше

После этого можно приступить к подбору значения *C*. Будем перебирать значения по логарифмической шкале, а именно из множества [0.001, 0.01, 0.1, 1.0, 10]. В качестве оптимизируемой метрики выберем *AUC-ROC*.

В sklearn для логистической регрессии это можно сделать, например, следующим способом: воспользоваться классом [LogisticRegressionCV](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegressionCV.html). В этом случае вам необходимо задать следующие параметры:
   - *Cs* — массив со значениями перебираемых параметров
   - *cv* — схему кросс-валидации, с помощью которой будет осуществляться выбор наилучшего
   - *scoring* — название способа оценивания качества; в данном случае это *roc_auc*
   - *random_state*
   
Чтобы подобрать оптимальное *C*, вызовите метод *fit*. 

**(2 балла)** Создайте объект класса [LogisticRegressionCV](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegressionCV.html), передав в качестве значения параметра *Cs* список [0.001, 0.01, 0.1, 1.0, 10], а в качестве *cv* — созданный ранее объект *KFold*. Обучите данный классификатор. Каким получилось оптимальное значение *C*?

Теперь создайте объект класса [LogisticRegression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) со значением *C* равным оптимальному, найденному выше. Обучите его (используя метод *fit*), на всей обучающей выборке.

Теперь пришло время посмотреть как полученный классификатор работает на тестовой выборке. 

**(2 балла)** Для оценивания качества построим *ROC* и *Precision-Recall* кривые, а так же посчитаем площадь под ними. Для этого вам нужно сделать следующее:
 - получите предсказание вероятности принадлежности положительному классу, используя метод *predict_proba* для тестовой выборки (обратите внимание, что в данном случае этот метод вернет 2 колонки — вероятности принадлежностей каждому классу, вам необходима первая, если считать с нуля).
 - постройте график, на котором изобразите *Precision-Recall* кривую, используя функцию [precision_recall_curve](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_curve.html). Более подробно как построить такой график было рассказано на одном из семинаров. Вычислите площадь под этим графиком.
 - проделайте аналогичные действия для *ROC*-кривой (воспользовавшись функцией [roc_curve](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_curve.html#sklearn.metrics.roc_curve)).

**(1 балл)** Теперь найдите, какую точность может иметь данный классификатор, чтобы полнота была не менее 0.8. Аналогично найдите какую максимальную полноту будет иметь классификатор при точности не менее 0.9.

А теперь вернемся к исходной многоклассовой задаче. Как известно, удобство sklearn заключается в том, что вам не нужно делать каких-либо модификаций алгоритмов в многоклассовом случае, так как все уже реализовано внутри библиотеки.

**(0.5 балла)** Обучите на исходной обучающей выборке логистическую регрессию. Постройте предсказания для тестовой выборки, используя метод *predict*. Посчитайте [accuracy](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html) на тестовой выборке.

**(1.5 балла)** Самое интересное — посмотреть на ошибки классификации. Для этого постройте [confusion_matrix](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html). Какие цифры путаются чаще всего между собой? Есть ли цифра, на которой на тестовой выборке классификатор ни разу не ошибся?